<a href="https://colab.research.google.com/github/abbeymars/AbigailMarsella-AER850Project3/blob/main/Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
from google.colab import drive
import cv2
import numpy as np
import matplotlib.pyplot as plt

#Step 1: Object Masking
drive.mount('/content/drive') #mount google drive
#Step 1.1 Upload Image
image_path = "/content/drive/My Drive/AER850Proj3/motherboard_image.JPEG"





Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(motherboard_image.JPEG).
